In [1]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from keras_vggface.vggface import VGGFace
from keras_vggface import utils

2022-11-25 12:33:47.856170: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# create a vggface model
model = VGGFace(model='vgg16')
model.summary()
# model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3), pooling='avg')

Metal device set to: AMD Radeon Pro 5500M


2022-11-25 12:33:51.639367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 12:33:51.640127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-25 12:33:51.640174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 12:33:51.640491: I tensorflow/core/common_runtime/pluggable_device/plug

Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)     

In [3]:
# summarize input and output shape
layer_name = 'fc8'
out = model.get_layer(layer_name).output
model = Model(model.input, out)
print('Inputs: %s' % model.inputs)
print('Outputs: %s' % model.outputs)
target_shape = (224, 224)

Inputs: [<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>]
Outputs: [<KerasTensor: shape=(None, 2622) dtype=float32 (created by layer 'fc8')>]


In [4]:
db_path = '/Users/santhoshnarayanan/Documents/Turing/GPU/VGGData/testset'
celebs  = []
 
for r, d, f in os.walk(db_path): # r=root, d=directories, f = files
    for file in f:
        if ('.jpg' in file):
            exact_path = r + "/" + file
            celebs.append(exact_path)

In [5]:
imgs = [image.load_img(img, target_size=target_shape) for img in celebs]
X = np.float32([utils.preprocess_input(np.float32(img), version=1) for img in imgs])
display(X.shape)

(8000, 224, 224, 3)

In [14]:
repA = model.predict(X[0:4000], batch_size=64)
display(repA.shape)
repB = model.predict(X[4000:8000], batch_size=64)
display(repB.shape)
rep = np.concatenate((repA, repB), axis=0)
display(rep.shape)

63/63 [==============================] - 79s 1s/step


(4000, 2622)

In [17]:
df = pd.DataFrame(rep)  
df.insert(loc = 0,
          column = 'celeb',
          value = celebs)
display(df)

,celeb,0,1,2,3,4,5,6,7,8,...,2612,2613,2614,2615,2616,2617,2618,2619,2620,2621
0,/Users/santhoshnarayanan/Documents/Turing/GPU/...,2.594817,-0.434783,0.606621,3.266035,0.597103,0.600513,0.553012,1.848994,1.382500,...,0.240451,-2.922260,-2.544503,-0.245354,-1.331547,-2.866831,-3.229215,1.122466,0.182207,-0.755669
1,/Users/santhoshnarayanan/Documents/Turing/GPU/...,4.750790,5.448501,3.474746,0.601897,-0.498479,4.499108,2.579011,3.014450,2.216500,...,-0.711684,0.259915,-2.627894,2.191211,-2.450251,-1.941559,-4.495428,-1.467024,1.814163,4.478118
2,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-1.341132,-0.609385,-2.179606,-1.865505,0.425404,-0.199042,-0.385628,1.690739,2.659145,...,-2.399040,0.780722,-2.663496,3.260226,1.200716,2.132750,1.412410,-4.562939,1.582824,4.562842
3,/Users/santhoshnarayanan/Documents/Turing/GPU/...,4.770988,-0.243045,2.897161,3.664442,2.233140,1.924740,0.763086,1.576417,2.291085,...,-0.259052,-1.911187,-2.592706,-1.359860,1.776673,-0.531952,-1.619988,-4.511042,-0.369517,0.393388
4,/Users/santhoshnarayanan/Documents/Turing/GPU/...,4.272125,-0.029436,-1.333247,1.142551,1.687836,0.479853,1.028407,4.002062,2.825842,...,-1.489381,-1.539137,-1.731887,-1.743123,-1.382905,-2.189632,-2.511028,0.873662,1.694410,0.746357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-1.207366,2.731673,-3.874018,-1.336324,-3.074984,-1.517171,2.307636,0.762435,3.631700,...,-2.840803,3.292569,2.218327,3.362937,2.148040,0.198981,-2.201727,-2.777040,8.273804,6.484615
7996,/Users/santhoshnarayanan/Documents/Turing/GPU/...,8.765278,10.337363,9.235224,1.467477,6.949498,3.644549,4.231451,7.804096,3.244748,...,-2.637619,-3.443918,-4.422450,-1.388282,0.481939,-5.317516,-4.708521,-5.240987,-3.128758,4.211435
7997,/Users/santhoshnarayanan/Documents/Turing/GPU/...,1.069455,-0.432792,1.427405,2.181358,0.294008,3.443064,1.722770,4.330018,-0.317990,...,-2.676923,-4.648773,-1.593403,0.832667,1.776019,1.521085,1.532222,2.159128,3.509219,4.177240
7998,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.113445,0.156092,0.584193,0.379440,-1.961920,1.778721,1.374092,3.356516,1.222736,...,-3.187177,-0.175232,-4.768421,0.925566,2.913509,8.612916,-2.260141,-1.507633,2.332282,4.463278


In [18]:
df.to_csv('rep_VGG16.csv')